In [2]:
import re
import sys
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

sys.path.append("../") 
from personal_library.NLP.preprocess import (
    remove_pattern,
    rm_pun_num_esp_cha,
    rm_length_word,
    tokenize,
    stemmer,
    join_tokenize,
    hashtag_extract,
    count_caps,
    hashtag_rm
)

from personal_library.NLP.data_analysis import(
    plot_labels_wordcloud,
    plot_hashtag_hist,
)

from personal_library.NLP.core.model_preprocessors import (
    corpus2vec,
    standard_word2vec_size
)

from gensim.models import KeyedVectors
from sklearn.manifold import TSNE

In [3]:
# Path to test file
test_path = '../data/test_tweets_anuFYb8.csv'
test = pd.read_csv(test_path)
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [9]:
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['tweet'], "@[\w]*")
test['tidy_tweet'] = rm_pun_num_esp_cha(test['tidy_tweet'])
test['tidy_tweet'] = rm_length_word(test['tidy_tweet'])
tokenized_tweet = tokenize(test['tidy_tweet'])
# tokenized_tweet = stemmer(tokenized_tweet)
test['tidy_tweet'] = join_tokenize(tokenized_tweet)
test['hashtag'] = hashtag_extract(test['tidy_tweet'], flatten=False)
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['tidy_tweet'], "#[\w]*")

In [18]:
tokenized_tweet = tokenize(test['tidy_tweet'])
test.fillna('test', inplace = True)
test["Name Length"] = test['tidy_tweet'].str.len()
test.head()

,id,tweet,tidy_tweet,hashtag,Name Length
0,31963,#studiolife #aislife #requires #passion #dedic...,find,"[studiolife, aislife, requires, passion, dedic...",11
1,31964,@user #white #supremacists want everyone to s...,want everyone here,"[white, supremacists, birds, movie]",22
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways heal your,"[acne, altwaystoheal, healthy, healing]",23
3,31966,is the hp and the cursed child book up for res...,cursed child book reservations already where w...,"[harrypotter, pottermore, favorite]",52
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",amazing hilarious ahmir uncle dave loves misses,"[bihday, nephew]",49


In [28]:
print("Median: ", test["Name Length"].median())
print("Media: ", test["Name Length"].mean())
print("Std: ", test["Name Length"].std())
print("MAx: ", test["Name Length"].max())

Median:  33.0
Media:  37.30923998371809
Std:  21.972422741175357
MAx:  113


In [15]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# loading
with open('../model_wehigts/tokenizer.pickle', 'rb') as handle:
    tk = pickle.load(handle)

max_len = 120
train_tokenized = tk.texts_to_sequences(test['tidy_tweet'])
x_test = pad_sequences(train_tokenized, maxlen=max_len)

print(x_test.shape)

(17197, 19)
